In [2]:
import tensorflow as tf

In [13]:
train=r"/home/teaching/Atmega_MIC/DP Students/Abhay/cropwise_data/tomato"
training_set = tf.keras.utils.image_dataset_from_directory(
    train,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=64,
    image_size=(224, 224),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 83019 files belonging to 10 classes.


In [14]:
classes = training_set.class_names
classes

['tomato_bacterial_spot',
 'tomato_early_blight',
 'tomato_healthy',
 'tomato_late_blight',
 'tomato_leaf_mold',
 'tomato_mosaic_virus',
 'tomato_septoria_leaf_spot',
 'tomato_spider_mites',
 'tomato_target_spot',
 'tomato_yellow_leaf_curl_virus']

In [15]:
total_batches = tf.data.experimental.cardinality(training_set).numpy()
train_size = int(0.8 * total_batches)
val_size = total_batches - train_size
train_dataset = training_set.take(train_size)
val_dataset = training_set.skip(train_size)
print(len(training_set))
print(len(train_dataset))
print(len(val_dataset))

1298
1038
260


In [17]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import callbacks

base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(10, activation='softmax')  # 6 classes
])

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])


In [ ]:
early_stop = callbacks.EarlyStopping(patience=3, restore_best_weights=True)
model.fit(train_dataset,
          validation_data=val_dataset,
          epochs=15,
          callbacks=[early_stop])